In [15]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [16]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [19]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Given the following JSON object representing a {category} component in a Revit architectural project, give a short 1-sentence description of the component"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{json_object}")]
)

chain = prompt_template | model

response = chain.invoke({"category": "Doors",
                         "json_object": "{'IfcGUID': '2wm6PO6TX9IxXwLP21HyWr', 'Host Id': 'Wall', 'Family and Type': 'Family Type: 2.027 x 0.945, Family: Pocket_Slider_Door_5851', 'Materials': ['Door - Frame', 'Door - Panel'], 'Head Height': '2027.0', 'Sill Height': '0.0', 'Area': '3.5520497174451937', 'Volume': '0.0913121749999984', 'Category': 'Doors', 'Level': 'Level(Name=Level 2, Elevation=3000)', 'Comments': 'None'}"})

print(response)


content='This component is a pocket sliding door with dimensions of 2.027 meters in height and 0.945 meters in width, located on Level 2 of the architectural project.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 183, 'total_tokens': 219, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_3de1288069', 'finish_reason': 'stop', 'logprobs': None} id='run-4afc79e2-64b7-4db9-a65e-f3b318b46757-0' usage_metadata={'input_tokens': 183, 'output_tokens': 36, 'total_tokens': 219, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
